Consult this site before:

https://stable-baselines3.readthedocs.io/en/master/guide/rl.html

https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html#a-taxonomy-of-rl-algorithms

# 1. Import dependencies

In [44]:
import os
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold
from stable_baselines3 import DQN

# 2. Load Environment

In [2]:
environment_name = "CartPole-v0"

In [3]:
environment_name

'CartPole-v0'

In [4]:
env = gym.make(environment_name)

In [5]:
env.reset()

array([0.03239708, 0.02701281, 0.03878179, 0.03278226], dtype=float32)

In [6]:
# Number of actions
env.action_space

Discrete(2)

In [7]:
episodes = 5
for episode in range(1, episodes+1):
    print(episode)

1
2
3
4
5


In [8]:
env.step(1)

(array([ 0.03293733,  0.22155777,  0.03943744, -0.24741678], dtype=float32),
 1.0,
 False,
 {})

In [9]:
# if you want to see the episodes in slow, comment the env.close() line
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:14.0
Episode:2 Score:44.0
Episode:3 Score:45.0
Episode:4 Score:46.0
Episode:5 Score:14.0


# Understanding The Environment
About the env: https://github.com/openai/gym/blob/master/gym/envs/classic_control/cartpole.py

Types of Algorithms for RL: https://spinningup.openai.com/en/latest/spinningup/rl_intro2.html

RL Algorithms that are implemented in the Stable Baselines3 project and some useful characteristics : https://stable-baselines3.readthedocs.io/en/master/guide/algos.html

In [10]:
# 0-push cart to left, 1-push cart to the right
env.action_space.sample()

1

In [11]:
# [cart position, cart velocity, pole angle, pole angular velocity]
env.observation_space.sample()

array([ 2.2092938e+00, -3.2156823e+38,  3.8371283e-01,  1.5788512e+38],
      dtype=float32)

# 3. Train an RL Model

In [12]:
#Make Your Directories First
log_path = os.path.join('Training', 'Logs')

In [13]:
log_path

'Training\\Logs'

In [14]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [15]:
# if you want to learn more about the model
PPO??

Init signature:
PPO(
    policy: Union[str, Type[stable_baselines3.common.policies.ActorCriticPolicy]],
    env: Union[gym.core.Env, stable_baselines3.common.vec_env.base_vec_env.VecEnv, str],
    learning_rate: Union[float, Callable[[float], float]] = 0.0003,
    n_steps: int = 2048,
    batch_size: int = 64,
    n_epochs: int = 10,
    gamma: float = 0.99,
    gae_lambda: float = 0.95,
    clip_range: Union[float, Callable[[float], float]] = 0.2,
    clip_range_vf: Union[NoneType, float, Callable[[float], float]] = None,
    normalize_advantage: bool = True,
    ent_coef: float = 0.0,
    vf_coef: float = 0.5,
    max_grad_norm: float = 0.5,
    use_sde: bool = False,
    sde_sample_freq: int = -1,
    target_kl: Optional[float] = None,
    tensorboard_log: Optional[str] = None,
    policy_kwargs: Optional[Dict[str, Any]] = None,
    verbose: int = 0,
    seed: Optional[int] = None,
    device: Union[torch.device, str] = 'auto',
    _init_setup_model: bool = True,
)
Source:        
c

In [16]:
model.learn(total_timesteps=20000)

Logging to Training\Logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 561  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 725         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008153238 |
|    clip_fraction        | 0.104       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.00388    |
|    learning_rate        | 0.0003      |
|    loss                 | 7.23        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0162     |
|    value_loss           | 56.5        |
-----------------------------------------
---

# 4. Save and Reload Model

In [17]:
PPO_path = os.path.join('Training', 'Saved Models', 'PPO_Model_Cartpole')

In [18]:
model.save(PPO_path)

In [19]:
del model

In [20]:
PPO_path

'Training\\Saved Models\\PPO_Model_Cartpole'

In [21]:
model.learn(total_timesteps=20000)

NameError: name 'model' is not defined

In [ ]:
model = PPO.load(PPO_path, env=env)

# 5. Evaluation

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

c:\Users\mathe\AppData\Local\Programs\Python\Python310\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


# 6. Test Model

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action, _ = model.predict(obs) #Now using Model here!
        obs, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[200.]
Episode:4 Score:[200.]
Episode:5 Score:[200.]


In [ ]:
env.reset()

array([[ 0.01889658,  0.00065927, -0.02613226, -0.03727489]],
      dtype=float32)

In [ ]:
obs = env.reset()

In [ ]:
model.predict(obs)

(array([0], dtype=int64), None)

In [ ]:
env.action_space.sample()

0

In [ ]:
action, _ = model.predict(obs)

In [ ]:
env.step(action)

(array([[-0.04411511, -0.20216304, -0.00284291,  0.2567859 ]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{}])

In [ ]:
obs = env.reset()
while True:
    action, _ = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done: 
        print('info', info)
        break
env.close()

info [{'TimeLimit.truncated': True, 'terminal_observation': array([-0.00886858,  0.00548441,  0.01650615, -0.01496904], dtype=float32)}]


# 7. Viewing Logs in Tensorboard

In [22]:
log_path

'Training\\Logs'

In [23]:
training_log_path = os.path.join(log_path, 'PPO_1')

In [24]:
training_log_path

'Training\\Logs\\PPO_1'

In [26]:
#localhost:6006
!tensorboard --logdir={training_log_path}

^C


# 8. Adding a Callback to the Training Stage

In [28]:
save_path = os.path.join('Training', 'Saved Models')
log_path = os.path.join('Training', 'Logs')

In [31]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])

In [32]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)
eval_callback = EvalCallback(env, 
                             callback_on_new_best=stop_callback, 
                             eval_freq=10000, 
                             best_model_save_path=save_path, 
                             verbose=1)

In [33]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [34]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 855  |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 769          |
|    iterations           | 2            |
|    time_elapsed         | 5            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0074173985 |
|    clip_fraction        | 0.0938       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.686       |
|    explained_variance   | 0.00177      |
|    learning_rate        | 0.0003       |
|    loss                 | 6.89         |
|    n_updates            | 10           |
|    policy_gradient_loss | -0.0151      |
|    value_loss           | 53.1         |
----------------------------

c:\Users\mathe\AppData\Local\Programs\Python\Python310\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 200          |
|    mean_reward          | 200          |
| time/                   |              |
|    total_timesteps      | 10000        |
| train/                  |              |
|    approx_kl            | 0.0067974306 |
|    clip_fraction        | 0.0583       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.605       |
|    explained_variance   | 0.191        |
|    learning_rate        | 0.0003       |
|    loss                 | 31.9         |
|    n_updates            | 40           |
|    policy_gradient_loss | -0.0137      |
|    value_loss           | 69.3         |
------------------------------------------
New best mean reward!
Stopping training because the mean reward 200.00  is above the threshold 200


In [35]:
model_path = os.path.join('Training', 'Saved Models', 'best_model')
model = PPO.load(model_path, env=env)

In [37]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)
env.close()

# 9. Changing Policies

In [38]:
#Netword Arch - This is an example of specifying a different architecture
#for the different neural networks used in PPO
net_arch=[dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]

In [42]:
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log=log_path, policy_kwargs={'net_arch': net_arch})

Using cpu device


In [43]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 1012 |
|    iterations      | 1    |
|    time_elapsed    | 2    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 555         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.014170745 |
|    clip_fraction        | 0.197       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.681      |
|    explained_variance   | 0.00954     |
|    learning_rate        | 0.0003      |
|    loss                 | 3.55        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0197     |
|    value_loss           | 17.8        |
-----------------------------------------
---

# 10. Using an Alternate Algorithm

In [45]:
model = DQN('MlpPolicy', env, verbose = 1, tensorboard_log=log_path)

Using cpu device


In [46]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Training\Logs\DQN_1
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.941    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 332      |
|    time_elapsed     | 0        |
|    total_timesteps  | 124      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.872    |
| time/               |          |
|    episodes         | 8        |
|    fps              | 601      |
|    time_elapsed     | 0        |
|    total_timesteps  | 269      |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration_rate | 0.818    |
| time/               |          |
|    episodes         | 12       |
|    fps              | 745      |
|    time_elapsed     | 0        |
|    total_timesteps  | 384      |
----------------------------------
------------------------

In [47]:
dqn_path = os.path.join('Training', 'Saved Models', 'DQN_model')

In [48]:
model.save(dqn_path)

In [49]:
model = DQN.load(dqn_path, env=env)

In [54]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(9.5, 0.6708203932499369)

In [55]:
env.close()